In [3]:
import os
import shutil
import time
import numpy as np
import sys
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
from IPython.display import display, Markdown, HTML

In [2]:
# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath(os.path.join('..','src'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

# Developed components
from document_processor import DocumentProcessor
from vector_db import ChromaVectorStore
from retrieval import ChromaRetrievalSystem
from question_generator import QuestionGenerator
from intent_classifier import IntentClassifier
from intent_handler import IntentHandlerManager, SessionState

{'intent': 'start_review', 'text': 'Can you start the quiz?'}


In [4]:
class MockQuestionGenerator:
    """Simple mock for question generation."""
    def generate_question(self, topics=None, question_type=None, difficulty=None):
        topic_text = "on " + ", ".join(topics) if topics else "on general knowledge"
        
        if question_type == "multiple-choice":
            return {
                "question": f"Here's a {difficulty} multiple-choice question {topic_text}: What is the capital of France?",
                "options": ["London", "Berlin", "Paris", "Madrid"],
                "correct_answer": "Paris",
                "type": "multiple-choice"
            }
        else:
            return {
                "question": f"Here's a {difficulty} free-text question {topic_text}: Explain the concept of object-oriented programming.",
                "correct_answer": "Object-oriented programming is a programming paradigm based on the concept of objects.",
                "type": "free-text"
            }

class MockAnswerEvaluator:
    """Simple mock for answer evaluation."""
    def evaluate_answer(self, question, user_answer):
        # For multiple-choice, check exact match
        if question["type"] == "multiple-choice":
            is_correct = user_answer.lower() == question["correct_answer"].lower()
        else:
            # For free-text, just check if certain keywords are present
            keywords = ["object", "programming", "concept"]
            is_correct = any(keyword.lower() in user_answer.lower() for keyword in keywords)
            
        feedback = "Correct! Well done." if is_correct else f"Not quite. The correct answer is: {question['correct_answer']}"
        
        return {
            "is_correct": is_correct,
            "feedback": feedback,
            "question": question["question"],
            "user_answer": user_answer,
            "correct_answer": question["correct_answer"]
        }

class MockDocumentProcessor:
    """Simple mock for document processing."""
    def process_document(self, document_data):
        return True

In [15]:
# Initialize the classifier and handler manager
classifier = IntentClassifier()

# Initialize the handler manager with mock components
handler_manager = IntentHandlerManager(
    question_generator=MockQuestionGenerator(),
    answer_evaluator=MockAnswerEvaluator(),
    document_processor=MockDocumentProcessor()
)

In [6]:

def process_user_input(user_input, document_data=None):
    """Process user input through the classifier and handler."""
    
    # Classify the intent
    intent_data = classifier.classify(user_input)
    intent_type = intent_data["intent"]
    
    print(f"Classified Intent: {intent_type}")
    print(f"Intent Data: {intent_data}\n")
    
    # Add document data if provided
    if document_data:
        intent_data["document_data"] = document_data
    
    # Handle the intent
    response = handler_manager.handle_intent(intent_type, intent_data)
    
    print(f"Response: {response}\n")
    print("-" * 80)
    
    return response

2025-03-21 22:23:54,625 - intent_handler - INFO - Handling intent: document_upload


TEST CASE 1: Document Upload
Classified Intent: document_upload
Intent Data: {'intent': 'document_upload', 'text': "I'd like to upload a document for review"}

Response: {'text': 'Document uploaded and processed successfully. I can now review that content with you.'}

--------------------------------------------------------------------------------


{'text': 'Document uploaded and processed successfully. I can now review that content with you.'}

In [9]:
# %%
# Test document upload intent
print("TEST CASE 1: Document Upload")
process_user_input("I'd like to upload a document for review")

# %%
# Test setting question type
print("TEST CASE 2: Set Question Type")
process_user_input("Can we use multiple choice questions?")

# %%
# Test setting number of questions
print("TEST CASE 3: Set Number of Questions")
process_user_input("I want to do 10 questions")

# %%
# Test setting topic
print("TEST CASE 4: Set Topic")
process_user_input("Let's focus on programming concepts")

# %%
# Test setting difficulty
print("TEST CASE 5: Set Difficulty")
process_user_input("Make it medium difficulty")

# %%
# Test starting a review
print("TEST CASE 6: Start Review")
process_user_input("Let's start the review now")

# %%
# Test answering a question
print("TEST CASE 7: Answer Question")
process_user_input("The answer is Paris")

# %%
# Test checking status
print("TEST CASE 8: Check Status")
process_user_input("How am I doing so far?")

# %%
# Test stopping a review
print("TEST CASE 9: Stop Review")
process_user_input("Let's stop the review session")

# %%
# Test enabling speech
print("TEST CASE 10: Enable Speech")
process_user_input("Can you enable speech interaction?")

2025-03-21 22:24:24,644 - intent_handler - INFO - Handling intent: document_upload
2025-03-21 22:24:24,645 - intent_handler - INFO - Handling intent: set_question_type
2025-03-21 22:24:24,646 - intent_handler - INFO - Handling intent: set_num_questions
2025-03-21 22:24:24,647 - intent_handler - INFO - Handling intent: set_topic
2025-03-21 22:24:24,648 - intent_handler - INFO - Handling intent: set_difficulty
2025-03-21 22:24:24,649 - intent_handler - INFO - Handling intent: start_review
2025-03-21 22:24:24,649 - intent_handler - INFO - Handling intent: answer
2025-03-21 22:24:24,650 - intent_handler - INFO - Handling intent: review_status
2025-03-21 22:24:24,650 - intent_handler - INFO - Handling intent: start_review
2025-03-21 22:24:24,651 - intent_handler - INFO - Handling intent: enable_speech


TEST CASE 1: Document Upload
Classified Intent: document_upload
Intent Data: {'intent': 'document_upload', 'text': "I'd like to upload a document for review"}

Response: {'text': 'Document uploaded and processed successfully. I can now review that content with you.'}

--------------------------------------------------------------------------------
TEST CASE 2: Set Question Type
Classified Intent: set_question_type
Intent Data: {'intent': 'set_question_type', 'question_type': 'multiple-choice', 'text': 'Can we use multiple choice questions?'}

Response: {'text': "I'll use multiple-choice questions for our review."}

--------------------------------------------------------------------------------
TEST CASE 3: Set Number of Questions
Classified Intent: set_num_questions
Intent Data: {'intent': 'set_num_questions', 'num_questions': 10, 'text': 'I want to do 10 questions'}

Response: {'text': "I'll prepare 10 questions for our review session."}

---------------------------------------------

{'text': "Speech interaction is now enabled. I'll listen for your voice and respond with speech."}

In [10]:

def run_conversation_test(conversation):
    """Run a simulated conversation through the system."""
    print("=== CONVERSATION TEST ===\n")
    
    for i, user_input in enumerate(conversation):
        print(f"User: {user_input}")
        response = process_user_input(user_input)
        print(f"System: {response.get('text', 'No response text')}")
        print()

In [16]:
# Define a conversation flow
conversation = [
    "Start review",
    "Hi, I want to upload a PDF document for review",
    "Let's use free-text questions instead of multiple choice",
    "I'd like to focus on Python programming topics",
    "Make it challenging",
    "Let's just do 3 questions",
    "Start the review session",
    "Object-oriented programming is a paradigm that uses objects to model real world problems",
    "How am I doing so far?",
    "Let's end the review",
    "Thanks for the help!"
]


In [17]:
run_conversation_test(conversation)


2025-03-21 22:25:39,591 - intent_handler - INFO - Handling intent: start_review
2025-03-21 22:25:39,592 - intent_handler - INFO - Handling intent: document_upload
2025-03-21 22:25:39,593 - intent_handler - INFO - Handling intent: set_question_type
2025-03-21 22:25:39,594 - intent_handler - INFO - Handling intent: set_topic
2025-03-21 22:25:39,595 - intent_handler - INFO - Handling intent: set_difficulty
2025-03-21 22:25:39,595 - intent_handler - INFO - Handling intent: set_num_questions
2025-03-21 22:25:39,596 - intent_handler - INFO - Handling intent: start_review
2025-03-21 22:25:39,597 - intent_handler - INFO - Handling intent: answer
2025-03-21 22:25:39,597 - intent_handler - INFO - Handling intent: review_status
2025-03-21 22:25:39,597 - intent_handler - INFO - Handling intent: start_review
2025-03-21 22:25:39,598 - intent_handler - INFO - Handling intent: answer


=== CONVERSATION TEST ===

User: Start review
Classified Intent: start_review
Intent Data: {'intent': 'start_review', 'text': 'Start review'}

Response: {'text': 'Please upload some documents first so I have material to review with you.'}

--------------------------------------------------------------------------------
System: Please upload some documents first so I have material to review with you.

User: Hi, I want to upload a PDF document for review
Classified Intent: document_upload
Intent Data: {'intent': 'document_upload', 'text': 'Hi, I want to upload a PDF document for review'}

Response: {'text': 'Document uploaded and processed successfully. I can now review that content with you.'}

--------------------------------------------------------------------------------
System: Document uploaded and processed successfully. I can now review that content with you.

User: Let's use free-text questions instead of multiple choice
Classified Intent: set_question_type
Intent Data: {'intent

In [19]:
# %%
# Test handling unknown intent
print("ERROR TEST 1: Unknown Intent")
process_user_input("What's the weather like today?")

# %%
# Test answering when not in review
print("ERROR TEST 2: Answer Without Review")
process_user_input("The answer is 42")

# %%
# Test with invalid number of questions
print("ERROR TEST 3: Invalid Question Count")
process_user_input("I want 100 questions")

# %% [markdown]
# ## Testing with Custom Input

# %%
def interactive_test():
    """Allow for interactive testing with custom input."""
    print("Enter your messages below. Type 'quit' to exit.")
    
    while True:
        user_input = input("\nYour message: ")
        if user_input.lower() == 'quit':
            break
            
        response = process_user_input(user_input)
        print(f"System response: {response.get('text', 'No response text')}")

# %%
# Uncomment to run interactive testing
# interactive_test()

2025-03-21 22:27:04,533 - intent_handler - INFO - Handling intent: answer
2025-03-21 22:27:04,535 - intent_handler - INFO - Handling intent: answer
2025-03-21 22:27:04,535 - intent_handler - INFO - Handling intent: set_num_questions


ERROR TEST 1: Unknown Intent
Classified Intent: answer
Intent Data: {'intent': 'answer', 'answer': "What's the weather like today?", 'text': "What's the weather like today?"}

Response: {'text': "We're not currently in a review session. Would you like to start one?"}

--------------------------------------------------------------------------------
ERROR TEST 2: Answer Without Review
Classified Intent: answer
Intent Data: {'intent': 'answer', 'answer': 'The answer is 42', 'text': 'The answer is 42'}

Response: {'text': "We're not currently in a review session. Would you like to start one?"}

--------------------------------------------------------------------------------
ERROR TEST 3: Invalid Question Count
Classified Intent: set_num_questions
Intent Data: {'intent': 'set_num_questions', 'num_questions': 100, 'text': 'I want 100 questions'}

Response: {'text': 'Please choose a number of questions between 1 and 50.'}

----------------------------------------------------------------------